### 2.4. Does implementing momentum require trading lots of small stocks– thus causing even larger trading costs?
For regulatory and liquidity reasons, AQR is particularly interested in using larger stocks for their momentum baskets. (Though they will launch one product that focuses on medium-sized stocks.)

Use the data provided on both small-stock “winners”, $r_t^{momSU}$, and small-stock “losers”, $r_t^{momSD}$, to construct a small-stock momentum portfolio,

$$
r_t^{momS} = r_t^{momSU} - r_t^{momSD}
$$

Similarly, use the data provided to construct a big-stock momentum portfolio,

$$
r_t^{momB} = r_t^{momBU} - r_t^{momBD}
$$

In [32]:
import pandas as pd
import numpy as np
sb = pd.read_excel('momentum_data.xlsx', sheet_name='size_sorts (total returns)').set_index('Date')
sb.head()

,SMALL LoPRIOR,ME1 PRIOR2,SMALL HiPRIOR,BIG LoPRIOR,ME2 PRIOR2,BIG HiPRIOR
Date,,,,,,
1927-01-31,-0.0001,0.0369,0.0050,-0.0040,0.0006,0.0023
1927-02-28,0.0673,0.0635,0.0582,0.0655,0.0398,0.0445
1927-03-31,-0.0315,-0.0314,-0.0199,-0.0364,-0.0018,0.0225
1927-04-30,-0.0070,-0.0046,0.0348,-0.0272,0.0073,0.0181
1927-05-31,0.0250,0.1123,0.0664,0.0550,0.0504,0.0692


In [33]:
r_momS = sb['SMALL HiPRIOR'] - sb['SMALL LoPRIOR']
r_momB = sb['BIG HiPRIOR'] - sb['BIG LoPRIOR']

### (a)
Fill out Table 4 over the sample 1994-2025.

In [34]:
mom = pd.read_excel('momentum_data.xlsx', sheet_name='momentum (excess returns)').set_index('Date')
ff = pd.read_excel('momentum_data.xlsx', sheet_name='factors (excess returns)').set_index('Date')
ff[['UMD', 'Small stocks','Big stocks']] = pd.concat([mom, r_momS, r_momB], axis=1)
sub = ff[ff.index >= '01-01-1994']
ans = pd.DataFrame(index=['UMD', 'Small stocks', 'Big stocks'], columns=['mean', 'vol', 'Sharpe', 'skewness', 'corr to MKT', 'corr to HML'])

for ret in ['UMD', 'Small stocks', 'Big stocks']:
    ans.loc[ret, 'mean'] = sub[ret].mean() * 12
    ans.loc[ret, 'vol'] = sub[ret].std() * np.sqrt(12)
    ans.loc[ret, 'Sharpe'] = ans.loc[ret, 'mean'] / ans.loc[ret, 'vol']
    ans.loc[ret, 'skewness'] = sub[ret].skew()
    ans.loc[ret, 'corr to MKT'] = sub[[ret, 'MKT']].corr().iloc[0, 1]
    ans.loc[ret, 'corr to HML'] = sub[[ret, 'HML']].corr().iloc[0, 1]

ans = ans.rename(index={'UMD': 'All stocks'})
ans

,mean,vol,Sharpe,skewness,corr to MKT,corr to HML
All stocks,0.041315,0.166054,0.248803,-1.451926,-0.305551,-0.215138
Small stocks,0.061797,0.167692,0.368514,-1.813338,-0.309372,-0.138885
Big stocks,0.020773,0.18185,0.114229,-0.855287,-0.272719,-0.264906


### (b)
Is the attractiveness of the momentum strategy mostly driven by the small stocks? That is, does a momentum strategy in large stocks still deliver excess returns at comparable risk?

A momentum strategy in large stocks does not deliver similar excess returns at comparable risk. Looking at the Sharpe ratios for small vs. big stocks, we can clearly see that the Sharpe for small stocks of roughly $0.37$ is more than three times the Sharpe for big stocks of roughly $0.11$.